In [21]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import seaborn as sns

### Market Basket Analysis Dataset

In order to be able to make a well-founded judgement on the quality of the final association rules that will result from the application of the techniques seen in the Market Basket Analysis course, I have created a set of superficial transactions whose underlying dynamics I know by construction.

**Context:**

We are located in a bookshop that offers nine different types of books. Each entry in the **transactions** dataset records a transaction made by a customer of this bookshop.

Each customer behaves as follows:

1. He randomly goes to a book genre **M** in the bookshop as soon as he arrives. (He doesn't necessarily buy a book of genre M.)
3. The probability that he will buy a book genre **N** is inversely proportional to the distance between **M** and **N**, as defined by the order in **bookstore_genres**.
2. He buys a minimum of two books and a maximum of **max_nb_of_elements_in_transactions** books.
3. Each transaction has at most one genre.

In [5]:
bookstore_genres = ["fiction","poetry","history","biography","cooking","health","travel","language","humor"]
number_of_bookstore_genres = len(bookstore_genres)
number_of_transactions = 10000

probability_distributions = np.ones((number_of_bookstore_genres,number_of_bookstore_genres))

for i in range(number_of_bookstore_genres):
    for j in range(1,number_of_bookstore_genres):
        if i-j >= 0:
            probability_distributions[i][i-j] /= 2**j
        if i+j < number_of_bookstore_genres:
            probability_distributions[i][i+j] /= 2**j

probability_distributions /= probability_distributions.sum(axis=1)[:,None]


transactions = []
max_nb_of_elements_in_transactions = 5

for _ in range(number_of_transactions):
    first_item_idx_in_transaction = np.random.choice(number_of_bookstore_genres)
    nb_of_items_in_transaction = np.random.randint(2, max_nb_of_elements_in_transactions+1)

    transaction = np.random.choice(bookstore_genres,(nb_of_items_in_transaction),False, p=probability_distributions[first_item_idx_in_transaction])

    transactions.append(transaction)


transactions[0]

array(['history', 'cooking', 'health', 'biography', 'humor'], dtype='<U9')

### Market Basket Analysis

**Let's define some context :**

* Let $D^{m \times n}$ be a onehot encoded dataset with each column representing a unique object $O_{j}$ and each row representing an event $E_{i}$ s.t. :

    **$D_{ij}$** is True if the object $O_{j}$ appears in the event $E_{i}$, and False otherwise.

* Let $A$ be the antecedent and $B$ the consequent.  

* $A$ or $B$ are defined as a set of objects (called itemset) {$O_{j1}$, $O_{j2}$, ..., $O_{jk}$}, s.t. $O_{ji}$ represents some object in $D$ and $k >= 1$ is a natural number. 

* $A$ and $B$ are disjoint itemsets.

* $A \to B$ is an association rule. (the probabilistic equivalent of the "if-then" relation in logic)

#### Question 1

* **Support** : 
    + **$Support(A \to B)$** : This is the number of events in which the union of the set of objects in $A$ and the set of objects in $B$ (i.e. $A \cup B$ ) appears divided by the total number of events in $D$.
    + If we set $B$ to be $A$, (i.e. **$Support(A \to A)$** ), this is the number of events in which the set of objects in $A$ appear divided by the total number of events in $D$.
    + Support($A \to A$) is also written as Support($A$).
    + Suppport is a symmetric metric : $Support(A \to B) = Support(B \to A)$
* **Confidence** :
    + **$Confidence(A \to B) = \frac{Support(A \to B)}{Support(A)}$**
    + Confidence might be interpreted as follows : If $A$ appears in an event $E_{i}$ then what is the probability that $B$ appears in $E_{i}$.
* **Lift** :
    + **$Lift(A \to B) = \frac{Confidence(A \to B)}{Support(B)} = \frac{Support(A \to B)}{Support(B)Support(A)}$**
    + Lift is a metric that compare the observed frequency of the union $A \cup B$ in $D$ and the frequency of the union $A \cup B$ in $D$ if $A$ and $B$ are independent.
    + If $Lift = 1$, then $A$ and $B$ are independent, i.e. if $A$ appears in an event $E_{i}$, then it doesn't impact the probability of $B$ appearing in $E_{i}$ which stays the same : Support($B$).
    + If $Lift > 1$, then if $A$ appears in an event $E_{i}$, then it impact positively the probability of $B$ appearing in $E_{i}$. i.e. $P(B \in E_{i} ⎹ A \in E_{i}) > Support(B)$.
    + If $Lift < 1$, then if $A$ appears in an event $E_{i}$, then it impact negatively the probability of $B$ appearing in $E_{i}$. i.e. $P(B \in E_{i} ⎹ A \in E_{i}) < Support(B)$.
    + $Lift$ is a symmetric metric : $Lift(A \to B) = Lift(B \to A)$

* **Leverage** : 
    + **$Leverage(A \to B) = Support(A \to B) - Support(B)Support(A)$**
    + $Leverage$ is a very similar metric to $Lift$, but it uses a difference rather than a division, which changes the range and the interpretation of the outputs.
    + If $Leverage = 0$, then $A$ and $B$ are independent.
    + If $Leverage$ tends towards 1, $P(B \in E_{i} ⎹ A \in E_{i}) > Support(B)$.
    + If $Leverage$ tends towards -1, $P(B \in E_{i} ⎹ A \in E_{i}) < Support(B)$.
    + $Leverage$ is a symmetric metric : $Leverage(A \to B) = Leverage(B \to A)$
* **Conviction** :
    + **$Conviction(A \to B) = \frac{1 - Support(B)}{1 - Confidence(A \to B)} = \frac{Support(A) - Support(A)Support(B)}{Support(A) - Support(A \to B)}$**
    + If $Conviction = 1$, then $A$ and $B$ are independent.
    + The more the presence of A positively impacts the presence of B in an event, the more $Conviction$ tends towards infinity.
    + The more the presence of A negatively impacts the presence of B in an event, the more $Conviction$ tends towards 0.
    + $Conviction$ is not symmetric.

#### Question 2

#### 2.1

To better understand the usefulness of the Pruning concept, we need to start by understanding the general approach to Market Basket Analysis:

1. Given that there are $n$ objects in $D^{m \times n}$, generate the set of all itemsets : $S = \{s ⎹ s = \{O_{j1}, O_{j2}, ..., O_{jk}\}, \forall k \in [1,...,n]\}$

2. $\forall s \in S$, generate all possible pairs $P = \{(A,B) ⎹ $A \cup B = s$ and $A \cap B =\{\}\}$. 
(Reminder : $A$ is the antecedent and $B$ the consequent)

3. Given a pair $p \in P$, we can now compute different metrics on it and consider to keep it or not depending on a previously defined threshold.

Now, the notion of Pruning intervene because we generally can't compute the complete set $S$ because the size of $S$ is $2^n - 1$. 
In concrete terms, we apply the notion of Pruning if we use any technique that consists of calculating only a subset of $S$ : For example, we could consider calculating the subset of $S$ only for $k = 3$.

Another Pruning technique is the **Apriori algorithm** which rely on two main principles :
* Subsets of frequent sets are frequent.
* If $\{O_{j1}, O_{j2}, ..., O_{jk}\}$ is infrequent, so is $\{O_{j1}, O_{j2}, ..., O_{jk}, O_{j(k+1)}\}$.

Consequently :

1. We can avoid calculating all the sets of $S$ containing $O_{j}$ if $\{O_{j}\}$ is not frequent in $D$.
2. We can avoid calculating all the sets of $S$ containing $O_{j1},O_{j2}$ if $\{O_{j1},O_{j2}\}$ is not frequent in $D$. Even if, both $O_{j1}$ and $O_{j2}$ are independently frequent in $D$.
3. And so on...

#### 2.2 

**min_support = 0.1**

In [17]:
onehot_encoder = TransactionEncoder()
transactions_onehot_encoding = onehot_encoder.fit(transactions).transform(transactions)
df_transactions_onehot_encoding = pd.DataFrame(transactions_onehot_encoding, columns=onehot_encoder.columns_)

frequent_itemsets = apriori(df_transactions_onehot_encoding,min_support=0.1,use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.4424,(biography)
1,0.4698,(cooking)
2,0.2892,(fiction)
3,0.4575,(health)
4,0.4233,(history)
5,0.2862,(humor)
6,0.3632,(language)
7,0.3584,(poetry)
8,0.4196,(travel)
9,0.2226,"(biography, cooking)"


#### Question 3

In [19]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(biography),(cooking),0.4424,0.4698,0.2226,0.503165,1.071019,0.014760,1.067154,0.118919
1,(cooking),(biography),0.4698,0.4424,0.2226,0.473819,1.071019,0.014760,1.059711,0.125065
2,(biography),(fiction),0.4424,0.2892,0.1496,0.338156,1.169279,0.021658,1.073968,0.259634
3,(fiction),(biography),0.2892,0.4424,0.1496,0.517289,1.169279,0.021658,1.155143,0.203675
4,(health),(biography),0.4575,0.4424,0.1865,0.407650,0.921452,-0.015898,0.941336,-0.135794
...,...,...,...,...,...,...,...,...,...,...
103,"(travel, language)",(humor),0.2337,0.2862,0.1457,0.623449,2.178368,0.078815,1.895626,0.705912
104,"(humor, language)",(travel),0.2030,0.4196,0.1457,0.717734,1.710520,0.060521,2.056216,0.521182
105,(travel),"(humor, language)",0.4196,0.2030,0.1457,0.347235,1.710520,0.060521,1.220961,0.715683
106,(humor),"(travel, language)",0.2862,0.2337,0.1457,0.509085,2.178368,0.078815,1.560961,0.757832


Support / Lift - Scatterplot